In [16]:
import pandas as pd
import numpy as np
import env
import os
from pydataset import data
from scipy import stats
import math

# import visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Remove limits on viewing dataframes
pd.set_option('display.max_columns', None)

1. Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [17]:
df = pd.read_csv('attendance.csv')
df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [18]:
df.columns

Index(['Unnamed: 0', '2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08'],
      dtype='object')

In [19]:
df = df.melt(id_vars='Unnamed: 0')
df.head()

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [20]:
df = df.rename(columns={'Unnamed: 0':'name', 'variable':'date', 'value':'status'})
df.head()

,name,date,status
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [21]:
df['attend_score'] = df.status.map({'P': 1, 'A': 0, 'T': 0.9, 'H': 0.5})
df.head()

,name,date,status,attend_score
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [22]:
df.groupby('name').attend_score.mean()

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attend_score, dtype: float64

2. Coffee Levels

a. Read the coffee_levels.csv file.

In [77]:
df = pd.read_csv('coffee_levels.csv')
df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


b. Transform the data so that each carafe is in it's own column.

In [78]:
df=df.pivot(index='hour', columns='coffee_carafe', values='coffee_amount').reset_index()
df.head()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


c. Is this the best shape for the data?

No, the original df where each observation (coffee level) is it's own row is the better shape.

3. Cake Recipes

a. Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [80]:
df = pd.read_csv('cake_recipes.csv')
df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


b. Tidy the data as necessary.

In [81]:
df[['recipe', 'position']] = df['recipe:position'].str.split(':',expand=True)
df.head()

,recipe:position,225,250,275,300,recipe,position
0,a:bottom,61.738655,53.912627,74.414730,98.786784,a,bottom
1,a:top,51.709751,52.009735,68.576858,50.228470,a,top
2,b:bottom,57.095320,61.904369,61.196980,99.248541,b,bottom
3,b:top,82.455004,95.224151,98.594881,58.169349,b,top
4,c:bottom,96.470207,52.001358,92.893227,65.473084,c,bottom


In [82]:
df = df.drop(columns='recipe:position')
df.head()

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom


In [83]:
df = df.melt(id_vars=['recipe','position'], var_name='temperature', value_name='taste_score')
df.head()

,recipe,position,temperature,taste_score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


c. Which recipe, on average, is the best? recipe b

In [84]:
df.groupby('recipe').taste_score.mean().sort_values(ascending=False).idxmax()

'b'

d. Which oven temperature, on average, produces the best results? 275

In [85]:
df.groupby('temperature').taste_score.mean().sort_values(ascending=False).idxmax()

'275'

e. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [86]:
df.groupby(['recipe','position','temperature']).taste_score.mean().sort_values(ascending=False).idxmax()

('b', 'bottom', '300')